# Assignment 2 of Research Track2 course

## User Interface node

In this notebook is implemented the code of the user interface in order to control the behavior of a robot. In order to do that this node interacts with nodes ***go_to_point.py*** and ***state_machine.cpp*** and ***position_service.cpp*** which are implemented in the package ***rt2_assignment1*** available at this link https://github.com/AliceNardelli/rt2_assignment1 in the branch action. In particular in this node are used different widgets such as buttons and sliders to pilot the robot and control its velocity.
To conclude different method of visualization of data such as figures and barplot are used to clearly understand robot behavior.

## How to run the code

Run inside a terminal and run:

**roslaunch rt2_assignment1 Newsimjup.launch**

In the meanwhile run this jupiter notebook.

Belows are imported needed headers and intialised  global variables

In [1]:

from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import time
import math
from rt2_assignment1.srv import Command
import actionlib
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
import ipywidgets as widgets
from IPython.display import display
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from rt2_assignment1.msg import GoalReachingActionResult
%matplotlib widget

#node initialized
rospy.init_node('user_interface')
#client of the user_interface to start the robot
ui_client = rospy.ServiceProxy('/user_interface', Command)
#client of the action to cancel the goal
client = actionlib.SimpleActionClient('go_to_point', rt2_assignment1.msg.GoalReachingAction)
#publisher of /cmd_vel to control robot's velocity
pub=rospy.Publisher('/cmd_vel', Twist, queue_size=1)
time.sleep(10)

#global var
cancelled=0 
reached=0 
active=False
start_time=0
stop_time=0
data = list()



## Start/Stop go to point via buttons

In the cell below are implemented two button to start/stop the robot go_to_point behavior. To note that the service client is called inside the callback of the start button, whereas inside the callback of the stop button are called **/cmd_vel** publisher to stop the robot and the action client to cancel the goals.

In [2]:

button_start = widgets.Button(description="START",layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='green'))

button_stop = widgets.Button(description="STOP",  layout=widgets.Layout(width='auto', grid_area='b2'),
                    style=widgets.ButtonStyle(button_color='red'))

def clicked_start(b): 
     global start_time
     global active
     #call the service client
     ui_client("start")      
     active=True
     start_time=time.time() 
        
def clicked_stop(b):
     global cancelled
     global stop_time
     global active
     global data
     # call the action client
     client.cancel_all_goals()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y = 0
     twist_msg.angular.z = 0
     #publish a zero velocity
     pub.publish(twist_msg)
     ui_client("stop")
     if active==True:
         cancelled = cancelled + 1 
         active=False
         stop_time=time.time()
         data.append(stop_time-start_time)
 
# on click method: code inside the clbk is executed only once the button is clicked

button_start.on_click(clicked_start)
button_stop.on_click(clicked_stop)

In [3]:
#grid box is used to order buttons
widgets.GridBox(children=[button_start,button_stop],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto auto',
                grid_template_columns='16% 16% 16% 16% 16% 16% ',
                grid_template_areas='''"b1 b2 . . . "''')
               )

GridBox(children=(Button(description='START', layout=Layout(grid_area='b1'), style=ButtonStyle(button_color='g…

In the followed cell there is the subscriber of the **/go_to_point/result**. The clbk is executed only once a goal is reached. Inside the clbk are updated different variables used to visualize robot behavior.

In [4]:
def clb(msg):
  global reached
  global start_time,stop_time
  global take
  global data 
  stop_time=time.time()
  data.append(stop_time-start_time)
  start_time=time.time()
  reached = reached+1
  

jr.subscribe('/go_to_point/result',GoalReachingActionResult, clb)

## Set linear/angular velocity via sliders

There are inserted two slider through which the user can chose a suitable linear or angular velocity to pilot the robot

In [5]:
msg = Twist()

a = widgets.FloatSlider(description='linear velocity' ,min=-1.0, max=1.0)
a.style.handle_color = 'lightblue'
display(a)

b = widgets.FloatSlider(description='angular velocity',min=-1.0, max=1.0)
b.style.handle_color = 'lightblue'
display(b)


def on_value_change(change):
    global msg
    msg.linear.x = change['new']
    pub.publish(msg)
    
def on_valueang_change(change):
    global msg
    msg.angular.z = change['new']
    pub.publish(msg)

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

FloatSlider(value=0.0, description='linear velocity', max=1.0, min=-1.0, style=SliderStyle(handle_color='light…

FloatSlider(value=0.0, description='angular velocity', max=1.0, min=-1.0, style=SliderStyle(handle_color='ligh…

## Button controller of the robot

Through this five buttons the user can directly pilot the robot. In order to do that inside the callback of each button firstly is called **stop()** function in order to stop go_to_point behavior through the action client secondly is published a specific velocity. The values of velocity are set in Twist message according to the choose of the user.

In [6]:

button_f = widgets.Button(description="forward",layout=Layout(width='auto', grid_area='B1'),
style=ButtonStyle(button_color='lightblue'))

button_tl = widgets.Button(description="turn left",layout=Layout(width='auto', grid_area='B2'),
style=ButtonStyle(button_color='salmon'))

button_bw = widgets.Button(description="backward",layout=Layout(width='auto', grid_area='B3'),
style=ButtonStyle(button_color='lightblue'))

button_tr = widgets.Button(description="turn right",layout=Layout(width='auto', grid_area='B4'),
style=ButtonStyle(button_color='salmon'))

button_s = widgets.Button(description="stop",layout=Layout(width='auto', grid_area='B5'),
style=ButtonStyle(button_color='red'))



def stop():
     global cancelled
     global stop_time
     global take,active
     global data
     client.cancel_all_goals()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y = 0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
     ui_client("stop")
     if active==True:
         cancelled = cancelled + 1 
         active=False
         stop_time=time.time()
         data.append(stop_time-start_time)
        
def clicked_f(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 1
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
        
def clicked_tl(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = 1
     pub.publish(twist_msg)
        
def clicked_tr(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = -1
     pub.publish(twist_msg)
        
def clicked_bw(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = -1
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
    
def clicked_s(b):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
    

            
            
button_f.on_click(clicked_f)
button_tr.on_click(clicked_tr)
button_bw.on_click(clicked_bw)
button_tl.on_click(clicked_tl)
button_s.on_click(clicked_s)



In [7]:
widgets.GridBox(children=[button_f, button_bw,button_tl,button_tr ,button_s],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto auto',
                grid_template_columns='16% 16% 16% 16% 16% 16% ',
                grid_template_areas='''". .  . B1  . ."". . B2 B5 B4 . "" . .  . B3 . . "''')
               )

GridBox(children=(Button(description='forward', layout=Layout(grid_area='B1', width='auto'), style=ButtonStyle…

## Reached and Cancelled goal visualization through barplot

In this cell is inserted a barplot to visualize reached and cancelled targets. The values **reached** is updated inside the callback of **/go_to_point/result** whereas the **cancelled** variable is increased each time action client is called (if reaching goal behavior is active)

In [ ]:
y = (reached,cancelled)
index = np.arange(2)

f1,axis2= plt.bar(index,y,width=0.5,color='b')
plt.ylabel('Scores')
plt.title('Reached and cancelled targets')
plt.xticks(index, ('reached', 'cancelled'))
plt.show()

## Mean time to reach a goal: histogram visualization

The histogram shows the mean time to reach a goal in go_to_point behavior. To compute the value a timer is set each time the robot starts to reach a goal and terminated once the goal is reached.

In [ ]:
X=list(range(0,len(data),1))
fig2=plt.subplot(111)
plt.bar(X,data,width=0.5,color='r')
plt.ylabel('time (s)')
plt.title('Mean time to reach a target')
plt.show()

## Figures with the animator

In the followed cell are represented firstly **/cmd_vel** and **/odom** linear velocity and angular velocity as lines. Values to show are continuously updated inside the callback whereas inside the animator figures are reshaped.

## Linear velocity

In [ ]:
#get the offset of time
clock1 = time.time() 
time1 = list(range(0, 10))
odom_linear =list()
cmdvel_linear=list()

In [ ]:
def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    return [line1,line2]

def clb_odom1(msg):
    if len(time1)>10:
      time1.pop(0)
     
    if len(odom_linear)>10:
      odom_linear.pop(0)
    #read velocity from /odom
    iupsilon = np.float32(msg.twist.twist.linear.x) 
    ix = np.float32(time.time()-clock1)
    #update the array
    odom_linear.append(iupsilon)
    time1.append(ix)
    
   

jr.subscribe('/odom',Odometry, clb_odom1)
    

In [ ]:
def clb_cmdvel1(msg):
    if len(cmdvel_linear)>10:
      cmdvel_linear.pop(0)
    #read velocity from /cmd_vel
    iupsilon = np.float32(msg.linear.x)
    #update array
    cmdvel_linear.append(iupsilon)
    

jr.subscribe('/cmd_vel', Twist, clb_cmdvel1)

In [ ]:
fig1, ax1 = plt.subplots()



line1, = ax1.plot([], [], color ='r', label ='/odom')
line2, = ax1.plot([], [], color ='b', label ='/cmd_vel')
ax1.set_xlabel('time (s)') 
ax1.set_ylabel('velocity m/s')
ax1.set_title("Linear velocity")
plt.legend(loc='upper right')

In [ ]:
def animate1(i):
    #updated values are set in the line
    line1.set_data(time1, odom_linear)
    line2.set_data(time1, cmdvel_linear)
    ax1.axis([time1[0]-0.2,time1[9]+0.2,-0.5,0.5]) #x axis is continously updated too   
    return [line1,line2]

#animator
anim1 = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

## Angular velocity

In [ ]:
#get the offset for time
clock2 = time.time() 
time2 = list(range(0, 10))
odom_angular =list()
cmdvel_angular=list()

In [ ]:
def init2():
    line3.set_data([], [])
    line4.set_data([], [])
    return [line3,line4]

def clb_odom2(msg):
    if len(time2)>10:
      time2.pop(0)
     
    if len(odom_angular)>10:
      odom_angular.pop(0)
    iupsilon = np.float32(msg.twist.twist.angular.z)
    ix = np.float32(time.time()-clock2)
    #actual data inserted
    odom_angular.append(iupsilon)
    time2.append(ix)
    
   

jr.subscribe('/odom',Odometry, clb_odom2)

In [ ]:
def clb_cmdvel2(msg):
    if len(cmdvel_angular)>10:
      cmdvel_angular.pop(0)
    iupsilon = np.float32(msg.angular.z)
    cmdvel_angular.append(iupsilon)
    

jr.subscribe('/cmd_vel', Twist, clb_cmdvel2)

In [ ]:
fig2, ax2 = plt.subplots()



line3, = ax2.plot([], [], color ='r', label ='/odom')
line4, = ax2.plot([], [], color ='b', label ='/cmd_vel')
ax2.set_xlabel('time (s)') 
ax2.set_ylabel('velocity rad/s')
ax2.set_title("Angular velocity")
plt.legend(loc='upper right')

In [ ]:

def animate2(i):
    line3.set_data(time2, odom_angular)
    line4.set_data(time2, cmdvel_angular)
    ax2.axis([time2[0]-0.2,time2[9]+0.2,-0.5,0.5])
    return [line3,line4]

anim2 = animation.FuncAnimation(fig2, animate2, init_func=init2,
                               frames=100, interval=20, blit=True)

## Position of the robot in the environment

Finally through the animator is plot the x and y position of the robot in the (x,y) plane. According to that it is possible to visualize how robot moves in the environment.

In [ ]:
x_data=list()
y_data=list()

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
ax.set_xlabel('x axis') 
ax.set_ylabel('y axis')
ax.set_title("Position")
line, = ax.plot([], [], 'ro')
    

In [ ]:
def init():
    line.set_data([], [])
    return (line,)


    
def odom_callback(msg):
    if len(x_data)>50:
      x_data.pop(0)
      
    if len(y_data)>50:
      y_data.pop(0)
    
    iupsilon = np.float32(msg.pose.pose.position.y)
    ix = np.float32(msg.pose.pose.position.x)
    y_data.append(iupsilon)
    x_data.append(ix)
    

    



jr.subscribe('/odom', Odometry, odom_callback)



In [ ]:
def animate(i):
    line.set_data(x_data, y_data)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

## Orientation of the robot

The orientation is shown as an arrow place where robot is in the environment and with the same orientation of the robot. **axis.quiver** allows to show the arrow, the orientation of the arrow is updated inside the animator.

In [ ]:
fig4, ax4 = plt.subplots()

ax4.quiver(0,0,dx,dy,label='Orientation')
ax4.set_title("Orientation")
ax4.legend()
line4, = ax.plot([], [], 'ro')

In [ ]:
def init4():
    ax4.arrow(0,0,dx,dy,label='Orientation')
    return (line4,)

def animate4(i):
    ax4.clear()
    ax4.quiver(x_data[48],y_data[48],x_data[49]-x_data[48],y_data[49]-y_data[48],label='Orientation')
    ax4.legend()
    ax4.set_title("Orientation")
    return (line4,)

anim4=animation.FuncAnimation(fig4, animate4, init_func=init4, frames=1, interval=100, blit=True)

>NOTE: doxygen documentation has been generated inside **action** branch 